In [5]:
from __future__ import unicode_literals, print_function, division
from io import open
import unicodedata
import string
import re
import random

import torch
import torch.nn as nn
from torch import optim
import torch.nn.functional as F
from torch.utils.data import DataLoader, Dataset

import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
import numpy as np

device = torch.device("cpu")


In [6]:
import time
import math

def asMinutes(s):
    m = math.floor(s / 60)
    s -= m * 60
    return '%dm %ds' % (m, s)

def timeSince(since, percent):
    now = time.time()
    s = now - since
    es = s / (percent)
    rs = es - s
    return '%s (- %s)' % (asMinutes(s), asMinutes(rs))


In [7]:
PAD_token = 2
SOS_token = 0
EOS_token = 1

class Lang:
    def __init__(self, name):
        self.name = name
        self.word2index = {}
        self.word2count = {}
        self.index2word = {2: "PAD", 0: "SOS", 1: "EOS"}
        self.n_words = 3  # Count PAD, SOS and EOS

    def addSentence(self, sentence, is_cn=False):
        for word in (sentence.split(' ') if not is_cn else list(sentence)):
            self.addWord(word)

    def addWord(self, word):
        if word not in self.word2index:
            self.word2index[word] = self.n_words
            self.word2count[word] = 1
            self.index2word[self.n_words] = word
            self.n_words += 1
        else:
            self.word2count[word] += 1
    def sentenceToIndexes(self, sentence):
        words = [word for word in sentence.split('')]
        indexes = [0, ]



In [8]:
# Turn a Unicode string to plain ASCII, thanks to
# https://stackoverflow.com/a/518232/2809427
def unicodeToAscii(s):
    return ''.join(
        c for c in unicodedata.normalize('NFD', s)
        if unicodedata.category(c) != 'Mn'
    )

# Lowercase, trim, and remove non-letter characters

def normalizeString(s, is_cn = False):
    
    if (is_cn):
        s = s.strip()
        s = re.sub(r"([。！？])", r" \1", s)
    else:
        s = unicodeToAscii(s.lower().strip())
        s = re.sub(r"([.!?])", r" \1", s)
        s = re.sub(r"[^a-zA-Z.!?]+", r" ", s)
    return s


In [9]:
def readLangs(lang1, lang2, reverse=False):
    print("Reading lines...")

    # Read the file and split into lines
    lines = open('data/%s-%s.txt' % (lang1, lang2), encoding='utf-8').\
        read().strip().split('\n')

    # Split every line into pairs and normalize
    pairs = [[normalizeString(s, i == 1 if reverse else i == 0) for i, s in enumerate(l.split('\t'))] for l in lines]

    # Reverse pairs, make Lang instances
    if reverse:
        pairs = [list(reversed(p)) for p in pairs]
        input_lang = Lang(lang2)
        output_lang = Lang(lang1)
    else:
        input_lang = Lang(lang1)
        output_lang = Lang(lang2)

    return input_lang, output_lang, pairs


In [10]:
seq_len = 50
batch_size = 1

In [11]:
MAX_LENGTH = seq_len

eng_prefixes = (
    "i am ", "i m ",
    "he is", "he s ",
    "she is", "she s ",
    "you are", "you re ",
    "we are", "we re ",
    "they are", "they re "
)

def filterPair(p):
    return len(list(p[0])) < MAX_LENGTH and \
        len(p[1].split(' ')) < MAX_LENGTH and \
        p[1].startswith(eng_prefixes)

def filterPairs(pairs):
    return [pair for pair in pairs if filterPair(pair)]


In [12]:
def prepareData(lang1, lang2, reverse=False):
    input_lang, output_lang, pairs = readLangs(lang1, lang2, reverse)
    print("Read %s sentence pairs" % len(pairs))
    pairs = filterPairs(pairs)
    print("Trimmed to %s sentence pairs" % len(pairs))
    print("Counting words...")
    for pair in pairs:
        input_lang.addSentence(pair[0], True)
        output_lang.addSentence(pair[1])
    print("Counted words:")
    print(input_lang.name, input_lang.n_words)
    print(output_lang.name, output_lang.n_words)
    return input_lang, output_lang, pairs

input_lang, output_lang, pairs = prepareData('en', 'cn', True)
print(random.choice(pairs))


Reading lines...
Read 29371 sentence pairs
Trimmed to 2294 sentence pairs
Counting words...
Counted words:
cn 1261
en 1533
['我累死了 。', 'i m dead tired .']


In [13]:
def indexesFromSentence(lang, sentence, is_cn=False):
    return [lang.word2index[word] for word in (sentence.split(' ') if not is_cn else list(sentence))]

def tensorFromSentence(lang, sentence, seq_len, is_cn=False):
    indexes = [SOS_token]
    indexes = indexes + indexesFromSentence(lang, sentence, is_cn)
    if(len(indexes) < seq_len -1):
       indexes = indexes + [EOS_token] + [PAD_token] * ((seq_len - 1) - len(indexes))
    elif(len(indexes) > seq_len):
        indexes = indexes[:seq_len -1]
        indexes.append(EOS_token)
    
    return torch.tensor(indexes, dtype=torch.long, device=device).view(-1, 1)

def tensorsFromPair(pair, seq_len=50):
    input_tensor = tensorFromSentence(input_lang, pair[0], seq_len, True)
    target_tensor = tensorFromSentence(output_lang, pair[1], seq_len)
    return (input_tensor, target_tensor)


In [14]:
class TranslationDataset(Dataset):
  def __init__(self, n_iters, seq_len):
    self.n_iters = n_iters
    self.pairs = [tensorsFromPair(random.choice(pairs), seq_len) for i in range(n_iters)]
    self.len = n_iters

  def __getitem__(self, index):
    return self.pairs[index]
  
  def __len__(self):
    return self.len

In [15]:
train_dataset = TranslationDataset(7500, seq_len)
test_dataset = TranslationDataset(1000, seq_len)
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

In [16]:
for item in train_loader:
    print(item[0].shape, item[1].shape)
    break

torch.Size([1, 50, 1]) torch.Size([1, 50, 1])


In [17]:
class Encoder(nn.Module):
    def __init__(self, input_size, hidden_size, dropout=0.1):
        super(Encoder, self).__init__()
        self.hidden_size = hidden_size

        self.embedding = nn.Embedding(input_size, hidden_size)
        self.gru = nn.GRU(hidden_size, hidden_size, batch_first=True)
        self.dropout = nn.Dropout(dropout)

    def forward(self, input):
        batch_size = input.size(0)
        hidden = torch.zeros(1, batch_size, self.hidden_size, device=device)
        embedded = self.dropout(self.embedding(input))
        embedded = embedded.squeeze(2)
        outputs, hidden = self.gru(embedded, hidden)

        return outputs, hidden


In [18]:
class Attention(nn.Module):
    def __init__(self, hidden_size):
        super().__init__()
        
        self.attn = nn.Linear(hidden_size * 2, hidden_size)
        self.v = nn.Linear(hidden_size, 1, bias = False)
        self.softmax = nn.Softmax(dim=1)
        
    def forward(self, hidden, encoder_outputs):
        batch_size = encoder_outputs.size(0)
        seq_len = encoder_outputs.size(1)
        
        hidden = hidden.permute(1, 0, 2)
        #repeat decoder hidden state src_len times, hidden = [batch size, seq len, hidden size]
        hidden = hidden.squeeze(0).repeat(1, seq_len, 1)
        
        # [batch size, seq len, hidden size * 2]
        outputs_cat_hidden = torch.cat((hidden, encoder_outputs), dim = 2)
        
        #energy = [batch size, src len, hidden size]
        energy = torch.tanh(self.attn(outputs_cat_hidden))

        #attention= [batch size, src len]
        attention = self.v(energy).squeeze(2)
        
        return self.softmax(attention)

In [19]:
class Decoder(nn.Module):
    def __init__(self, hidden_size, output_size, dropout=0.1):
        super().__init__()
        self.attention = Attention(hidden_size)
        self.hidden_size = hidden_size
        self.output_size = output_size
        self.embedding = nn.Embedding(output_size, hidden_size)
        self.rnn = nn.GRU(hidden_size * 2, hidden_size)
        self.out = nn.Linear(hidden_size * 3, output_size)
        self.dropout = nn.Dropout(dropout)
    
    def forward(self, input, hidden, encoder_outputs):
        #input = [batch size, 1]
        #hidden = [1, batch size, hidden_size]
        #encoder_outputs = [batch size, src len, hidden_size]
        
        #embedded = [1, batch size, hidden_size]
        embedded = self.dropout(self.embedding(input))
        embedded = embedded.permute(1, 0, 2)
        
        #a = [batch size, src len]
        a = self.attention(hidden, encoder_outputs)
        #a = [batch size, 1, src len]
        a = a.unsqueeze(1)

        #weighted = [batch size, 1, hidden_size]
        weighted = torch.bmm(a, encoder_outputs)
        
        #weighted = [1, batch size, hidden_size]
        weighted = weighted.permute(1, 0, 2)

        #rnn_input = [1, batch size, hidden_size * 2]
        rnn_input = torch.cat((embedded, weighted), dim = 2)

        #output = [seq len, batch size, hidden_size]
        #hidden = [n_layers, batch size, hidden_size]
        output, hidden = self.rnn(rnn_input, hidden)
        
        #seq len, n layers and n directions will always be 1 in this decoder, therefore:
        #output = [1, batch size, dec hid dim]
        #hidden = [1, batch size, dec hid dim]
        #this also means that output == hidden
        # assert (output == hidden).all()

        #embedded = [batch size, hidden_size]
        embedded = embedded.squeeze(0)
        #output = [batch size, hidden_size]
        output = output.squeeze(0)
        #weighted = [batch size, hidden_size]
        weighted = weighted.squeeze(0)
        
        #prediction = [batch size, output size]
        prediction = self.out(torch.cat((output, weighted, embedded), dim = 1))
        return prediction, hidden, a

In [20]:
class Seq2Seq(nn.Module):
    def __init__(self, input_size, hidden_size, output_size, device: torch.device):
        super().__init__()
        self.output_size = output_size
        self.encoder = Encoder(input_size, hidden_size).to(device)
        self.decoder = Decoder(hidden_size, output_size).to(device)
        self.device = device

    def forward(self, input, target, use_teacher_forcing=False):
        batch_size = target.size(0)
        target_length = target.size(1)

        encoder_outputs, encoder_hidden = self.encoder(input)

        decoder_input = target[:, 0,:]

        decoder_hidden = encoder_hidden #

        
        decoder_outputs = torch.zeros(target_length, batch_size, self.output_size, device=device)

        if use_teacher_forcing:
            # Teacher forcing: Feed the target as the next input
            for di in range(target_length):
                decoder_output, decoder_hidden, decoder_attention = self.decoder(decoder_input, decoder_hidden, encoder_outputs)
                decoder_outputs[di] = decoder_output
                if (di == target_length -1):
                    break
                decoder_input = target[:, di + 1,:]  # Teacher forcing

        else:
            # Without teacher forcing: use its own predictions as the next input
            for di in range(target_length):
                decoder_output, decoder_hidden, decoder_attention  = self.decoder(decoder_input, decoder_hidden, encoder_outputs)
                topv, topi = decoder_output.topk(1)
                decoder_input = topi.detach()  # detach from history as input

                decoder_outputs[di] = decoder_output
        
        return decoder_outputs.permute(1, 0 ,2)

    def interface(self, input, max_len):
        input_length = input.size(0)

        encoder_outputs, encoder_hidden = self.encoder(input.unsqueeze(0))

        decoder_input = torch.tensor([[SOS_token]])

        decoder_hidden = encoder_hidden #

        decoder_attentions = torch.zeros(max_len, input_length)
        decoded_words = []
        # Without teacher forcing: use its own predictions as the next input
        for di in range(max_len):
            decoder_output, decoder_hidden, decoder_attention  = self.decoder(decoder_input, decoder_hidden, encoder_outputs)
            decoder_attentions[di] = decoder_attention.squeeze().data
            topv, topi = decoder_output.topk(1)
            decoder_input = topi.detach()  # detach from history as input
            if topi.item() == EOS_token:
                decoded_words.append('<EOS>')
                break
            decoded_words.append(topi.item())
        return decoded_words, decoder_attentions

In [21]:
hidden_size = 256
model = Seq2Seq(input_lang.n_words, hidden_size, output_lang.n_words, device)
criterion = torch.nn.CrossEntropyLoss(ignore_index=PAD_token)
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)

In [22]:
def train(epoch, plot_every, print_every, device, teacher_forcing_ratio=0.5):
  plot_losses = []
  print_loss_total = 0  # Reset every print_every
  plot_loss_total = 0  # Reset every plot_every
  start = time.time()
  for iter, data in enumerate(train_loader, 1):
    inputs, target = data
    inputs, target = inputs.to(device), target.to(device)
    optimizer.zero_grad()

    use_teacher_forcing = True if random.random() < teacher_forcing_ratio else False
    outputs = model(inputs, target, use_teacher_forcing)
    outputs = outputs.reshape(-1, outputs.shape[-1])
    target = target.view(-1)
    loss = criterion(outputs, target)
    loss.backward()
    optimizer.step()

    print_loss_total += loss.item() / target.size(0)
    plot_loss_total += loss.item() / target.size(0)

    if iter % print_every == 0:
      print_loss_avg = print_loss_total / print_every
      print_loss_total = 0
      print('Epoch %d : %s (%d %d%%) %.4f' % (epoch, timeSince(start, iter / len(train_loader)), iter, iter / len(train_loader) * 100, print_loss_avg))

    if iter % plot_every == 0:
        plot_loss_avg = plot_loss_total / plot_every
        plot_losses.append(plot_loss_avg)
        plot_loss_total = 0
    
  return plot_losses

In [ ]:
train_losses = train(0, plot_every=100, print_every=1000, device=device, teacher_forcing_ratio=0.5)

In [ ]:
def showPlot(points):
    plt.figure()
    fig, ax = plt.subplots()
    # this locator puts ticks at regular intervals
    loc = ticker.MultipleLocator(base=0.2)
    ax.yaxis.set_major_locator(loc)
    plt.plot(points)

showPlot(train_losses)

In [ ]:
model_path = './models/seq2seq6.pt'

In [ ]:
torch.save(model.state_dict(), model_path)

In [ ]:
def evaluate(model, sentence, max_length=MAX_LENGTH):
    with torch.no_grad():
        input = tensorFromSentence(input_lang, sentence, seq_len, True)
        input_length = input.size()[0]

        decoded_words, attentions = model.interface(input, max_length)

        return decoded_words, attentions


In [ ]:
def evaluateRandomly(model, n=10):
    for i in range(n):
        pair = random.choice(pairs)
        print('>', pair[0])
        print('=', pair[1])
        output_words, _ = evaluate(model, pair[0])
        output_sentence = ' '.join(output_words)
        print('<', output_sentence)
        print('')


In [ ]:
model1 = Seq2Seq(input_lang.n_words, hidden_size, output_lang.n_words, device)

model1.load_state_dict(torch.load(model_path))

In [ ]:
evaluateRandomly(model1)

In [ ]:
output_words, attentions = evaluate(model1, "我不确定。")
output_sentence = ' '.join(output_words)
print(output_sentence)
plt.matshow(attentions.numpy())

In [ ]:
def showAttention(input_sentence, output_words, attentions):
    # Set up figure with colorbar
    fig = plt.figure()
    ax = fig.add_subplot(111)
    cax = ax.matshow(attentions.numpy(), cmap='bone')
    fig.colorbar(cax)

    # Set up axes
    ax.set_xticklabels([''] + list(input_sentence) +
                       ['<EOS>'], rotation=90)
    ax.set_yticklabels([''] + output_words)

    # Show label at every tick
    ax.xaxis.set_major_locator(ticker.MultipleLocator(1))
    ax.yaxis.set_major_locator(ticker.MultipleLocator(1))

    plt.show()

def evaluateAndShowAttention(input_sentence):
    output_words, attentions = evaluate(model1, input_sentence)
    print('input =', input_sentence)
    print('output =', ' '.join(output_words))
    showAttention(input_sentence, output_words, attentions)

evaluateAndShowAttention("我不确定。")